# Snowflake Queries - Tasty Bytes Data Analysis

This notebook executes all Snowflake queries against the Tasty Bytes database.

In [1]:
# Import required libraries
import pandas as pd
from connection import get_config
import snowflake.connector

In [2]:
# Establish Snowflake connection
cfg = get_config()

conn_params = {
    "account": cfg["account"],
    "user": cfg["user"],
    "password": cfg["password"],
    "role": cfg["role"],
    "database": cfg["database"],
    "schema": cfg["schema"],
}

if cfg["warehouse"]:
    conn_params["warehouse"] = cfg["warehouse"]

con = snowflake.connector.connect(**conn_params)
cur = con.cursor()

# If no warehouse was set, use the first available
if not cfg["warehouse"]:
    cur.execute("SHOW WAREHOUSES")
    warehouses = cur.fetchall()
    if warehouses:
        warehouse_name = warehouses[0][0]
        cur.execute(f"USE WAREHOUSE {warehouse_name}")

print("✓ Connected to Snowflake")

✓ Connected to Snowflake


## Query 1: Row Count in MENU Table

In [3]:
cur.execute("SELECT COUNT(*) AS row_count FROM menu;")
df1 = cur.fetch_pandas_all()
print("How many rows are in the MENU table?")
df1

How many rows are in the MENU table?


,ROW_COUNT
0,100


## Query 2: Top 10 Rows from MENU

In [4]:
cur.execute("SELECT TOP 10 * FROM menu;")
df2 = cur.fetch_pandas_all()
print("Top 10 rows from MENU table:")
df2

Top 10 rows from MENU table:


,MENU_ID,MENU_TYPE_ID,MENU_TYPE,TRUCK_BRAND_NAME,MENU_ITEM_ID,MENU_ITEM_NAME,ITEM_CATEGORY,ITEM_SUBCATEGORY,COST_OF_GOODS_USD,SALE_PRICE_USD,MENU_ITEM_HEALTH_METRICS_OBJ
0,10001,1,Ice Cream,Freezing Point,10,Lemonade,Beverage,Cold Option,0.65,3.5,"{\n ""menu_item_health_metrics"": [\n {\n ..."
1,10002,1,Ice Cream,Freezing Point,11,Sugar Cone,Dessert,Cold Option,2.50,6.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
2,10003,1,Ice Cream,Freezing Point,12,Waffle Cone,Dessert,Cold Option,2.50,6.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
3,10004,1,Ice Cream,Freezing Point,13,Two Scoop Bowl,Dessert,Cold Option,3.00,7.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
4,10005,1,Ice Cream,Freezing Point,14,Bottled Water,Beverage,Cold Option,0.50,2.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
5,10006,1,Ice Cream,Freezing Point,15,Bottled Soda,Beverage,Cold Option,0.50,3.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
6,10007,1,Ice Cream,Freezing Point,16,Ice Tea,Beverage,Cold Option,0.75,3.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
7,10008,1,Ice Cream,Freezing Point,17,Ice Cream Sandwich,Dessert,Cold Option,1.00,4.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
8,10009,1,Ice Cream,Freezing Point,18,Mango Sticky Rice,Dessert,Cold Option,1.25,5.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."
9,10010,1,Ice Cream,Freezing Point,19,Popsicle,Dessert,Cold Option,0.50,3.0,"{\n ""menu_item_health_metrics"": [\n {\n ..."


## Query 3: Freezing Point Menu Items

In [5]:
cur.execute("SELECT menu_item_name FROM menu WHERE truck_brand_name = 'Freezing Point';")
df3 = cur.fetch_pandas_all()
print("Menu items from Freezing Point brand:")
df3

Menu items from Freezing Point brand:


,MENU_ITEM_NAME
0,Lemonade
1,Sugar Cone
2,Waffle Cone
3,Two Scoop Bowl
4,Bottled Water
5,Bottled Soda
6,Ice Tea
7,Ice Cream Sandwich
8,Mango Sticky Rice
9,Popsicle


## Query 4: Mango Sticky Rice Profit

In [6]:
cur.execute("""
    SELECT
        menu_item_name,
        (sale_price_usd - cost_of_goods_usd) AS profit_usd
    FROM menu
    WHERE truck_brand_name = 'Freezing Point'
    AND menu_item_name = 'Mango Sticky Rice';
""")
df4 = cur.fetch_pandas_all()
print("Profit on Mango Sticky Rice:")
df4

Profit on Mango Sticky Rice:


,MENU_ITEM_NAME,PROFIT_USD
0,Mango Sticky Rice,3.75


## Query 5: Mango Sticky Rice Ingredients

In [7]:
cur.execute("""
    SELECT
        m.menu_item_name,
        obj.value:"ingredients"::ARRAY AS ingredients
    FROM menu m,
        LATERAL FLATTEN (input => m.menu_item_health_metrics_obj:menu_item_health_metrics) obj
    WHERE truck_brand_name = 'Freezing Point'
    AND menu_item_name = 'Mango Sticky Rice';
""")
df5 = cur.fetch_pandas_all()
print("Ingredients for Mango Sticky Rice:")
df5

Ingredients for Mango Sticky Rice:


,MENU_ITEM_NAME,INGREDIENTS
0,Mango Sticky Rice,"[\n ""Sweet Mango"",\n ""Sticky Salted Rice"",\n..."


## Close Connection

In [9]:
cur.close()
con.close()
print("✓ Connection closed")

✓ Connection closed
